In [1]:
# @title ##**Install modules** { display-mode: "form" }
from IPython.display import clear_output
!pip install youtube_dl
!pip install get-video-properties
!pip install kora
!pip install spleeter
!pip install gTTS==2.2.2
!pip install pydub
clear_output()

In [6]:
# @title ##**Select storage type** { display-mode: "form" }

from google.colab.output import eval_js
import os
import glob
from google.colab import drive
from IPython.display import clear_output
from subprocess import Popen, PIPE
import re
from videoprops import get_video_properties
import datetime
import shutil
from pydub import AudioSegment
from gtts import gTTS
import IPython.display as ipd
from pydub import AudioSegment
from IPython.display import HTML

#calculate video time
def calculate_video_duration(video_file):
  res = Popen(['ffmpeg', '-i', video_file, '-hide_banner'],stdout=PIPE,stderr=PIPE)
  none,meta = res.communicate()
  meta_out = meta.decode()
  #---| Take out info
  duration = re.search(r'Duration:.*', meta_out)
  my_time=duration.group().split(",")[0].split("Duration:")[-1]
  hour,minutes,seconds=my_time.split(":")
  video_in_seconds=float(hour)*60*60+float(minutes)*60+float(seconds)
  return video_in_seconds
def successful():
  eval_js('new Audio("https://www.myinstants.com/media/sounds/anime-wow-sound-effect.mp3").play()')    

storage="temporary"#@param["temporary","google drive"]
parent_directory=""
if storage=="google drive":
  drive.mount('/content/gdrive')
  parent_directory="/content/gdrive/MyDrive"
  clear_output()
  print("Using drive as a storage")
elif storage=="temporary":
  os.chdir("/content/")
  parent_directory=os.getcwd()
  print("Using colab as a temporary storage")  

try:
  os.mkdir("input_video")
except:
  pass
try:
  os.mkdir("output_video")    
except:
  pass

Using colab as a temporary storage


In [7]:
# @title ##**Choose youTube video quality** { display-mode: "form" }

test_youtube_link = "https://youtu.be/udeQhZHx-00" #@param {type:"string"}
youtube_quality_id=test_youtube_link.split("/")
check_all_quality=f"!youtube-dl --list-formats https://www.youtube.com/watch?v={youtube_quality_id[-1]} | grep mp4"
check_all_quality

'!youtube-dl --list-formats https://www.youtube.com/watch?v=udeQhZHx-00 | grep mp4'

#Copy the upper cell output and run in next cell

In [8]:
!youtube-dl --list-formats https://www.youtube.com/watch?v=udeQhZHx-00 | grep mp4

140          m4a        audio only tiny  129k , m4a_dash container, mp4a.40.2@129k (44100Hz), 10.40MiB
160          mp4        256x144    144p   35k , mp4_dash container, avc1.4d400c@  35k, 30fps, video only, 2.87MiB
133          mp4        426x240    240p   75k , mp4_dash container, avc1.4d4015@  75k, 30fps, video only, 6.07MiB
134          mp4        640x360    360p  138k , mp4_dash container, avc1.4d401e@ 138k, 30fps, video only, 11.11MiB
135          mp4        854x480    480p  234k , mp4_dash container, avc1.4d401f@ 234k, 30fps, video only, 18.81MiB
136          mp4        1280x720   720p  409k , mp4_dash container, avc1.64001f@ 409k, 30fps, video only, 32.86MiB
298          mp4        1280x720   720p60  565k , mp4_dash container, avc1.640020@ 565k, 60fps, video only, 45.42MiB
299          mp4        1920x1080  1080p60 1013k , mp4_dash container, avc1.64002a@1013k, 60fps, video only, 81.40MiB
18           mp4        640x360    360p  236k , avc1.42001E, 30fps, mp4a.40.2 (44100Hz), 

In [9]:
# @title ##**Download youtube video** { display-mode: "form" }

os.chdir(f"{parent_directory}/input_video")
os.system("rm -rf *")
enter_youtube_link = "https://youtu.be/udeQhZHx-00" #@param {type:"string"}
youtube_quality = "Auto"#@param ["Manual","Auto"]
quality_number = "299" #@param {type:"string"}
youtube_id=enter_youtube_link.split("/")
if youtube_quality=="Auto":
  dummy="youtube-dl --format mp4 https://www.youtube.com/watch?v="+youtube_id[-1]
  if os.system(dummy) ==0:
    print("Download successful")
  else:
    print("Can't download")
elif youtube_quality=="Manual":
  dummy=f"youtube-dl  https://youtu.be/{youtube_id[-1]} -f {quality_number}+bestaudio"
  if os.system(dummy) == 0:
    print("Download successful")
  else:
    print("Can't download")

file_types=["mp4","mkv","webm","mp4a","avi","mov"]
video_files=[]
for i in file_types:
  path=os.getcwd()+f"/*.{i}"
  for j in glob.glob(path):
    video_files.append(j)
the_video=video_files[0].split("/")[-1]    
if the_video.endswith(".mp4"):
      extension=".mp4"
elif the_video.endswith(".mp4"):
      extension=".mp4" 
elif the_video.endswith(".mkv"):
      extension=".mkv"
elif the_video.endswith(".webm"):
      extension=".webm" 
elif the_video.endswith(".mp4a"):
      extension=".mp4a"
elif the_video.endswith(".avi"):
      extension=".avi" 
elif the_video.endswith(".mov"):
      extension=".mov"   
os.rename(the_video,f"input_video{extension}")
f_name="input_video"  

props = get_video_properties(f"input_video{extension}")
print(f"Resolution: {props['width']}×{props['height']}")

#@title Format- H:M:S, example- 00:10:30
select_duration ="full video"#@param["full video","some part"]
enter_start_time= "00:00:00" #@param {type:"string"}
enter_end_time= "00:00:60" #@param {type:"string"}


start_time_float = enter_start_time.split(":")
end_time_float = enter_end_time.split(":")
start_time_in_second = (float(start_time_float[0])*60*60) + (float(start_time_float[1])*60) + float(start_time_float[2])
end_time_in_second = (float(end_time_float[0])*60*60) + (float(end_time_float[1])*60) + float(end_time_float[2])
cut_time_duration = end_time_in_second - start_time_in_second
cut_time = str(datetime.timedelta(seconds=cut_time_duration))

if  select_duration =="full video":
  video_duration=calculate_video_duration(f_name+f"{extension}")
elif select_duration =="some part":
    cut_command=f"ffmpeg -ss {enter_start_time} -i input_video{extension} -t {cut_time} -c copy cut{extension}"
    # cut_command=f"ffmpeg -i input_video{extension} -vcodec copy -acodec copy -ss {enter_start_time} -t {cut_time} cut{extension}"
    os.system(cut_command)
    os.remove(f"input_video{extension}")
    os.rename(f"cut{extension}",f"input_video{extension}")
    video_duration=calculate_video_duration(f_name+f"{extension}")
input_video_duration = str(datetime.timedelta(seconds=video_duration))

print(f"input video duration {input_video_duration}")
html_code="<h3>Watch original youtube video</h3>"
display(HTML(html_code))
youtube_video_display=""
youtube_video_display='<iframe width="'+str(560)+'" height="'+str(315)+'"\nsrc="https://www.youtube.com/embed/'+youtube_id[-1]+'"></iframe>'
display(HTML(youtube_video_display))
successful()


Download successful
Resolution: 1280×720
input video duration 0:11:13.400000


#Condition the video must have subtitle than download subtitle in your favorite  language.
visit [downsub.com](https://downsub.com/) to download your youtube videos subtitle in your language and copy the text

In [10]:
# @title ##**Paste the subtitle** { display-mode: "form" }

paste_subtitles = "It\u2019s bread because I\u2019ve been with you today.   Let's make a machine that makes rock, paper, scissors, scissors eyeballs using a media pipe.   The source code is on the Jackie Top, and I wrote it by 2 people and made it by referring to this code in Google Media Pipe Pose Station.   In order to explain the code written by 2 people more easily, I fork it like this to explain to you guys.   The code itself is very short because it consists of 77 lines. So, I will explain the simple principle of 105 machines that I am going to explain today.   First of all, if I explain the pipeline, it consists of 4 pipelines.   I learned a lot by looking at the code, especially in the gesture recognition part, that I can recognize it like this.   I learned it. First, the media pipe recognizes the achievements.   After implanting like this, the angle is calculated on the finger joint. There are many points of the sea of \u200B\u200Bthe finger with a lot of finger angle.   I named it Cooking Maker Int, but each of these points will have these angles   Nowadays and nowadays I calculated the angle. I keep it like this. I improve the angle of the vector vector.   It calculates the angle so that you can know how many times the finger is bent.   Thirdly, this angle was used to recognize gestures to protect it.   This is the rock, or the scissors are now learning. Finally, the winner is decided.   Mohyun's simple It's original, but since I used scissors, all the kids have a simple algorithm to determine the winner in this way   You can make a galvo machine through 4 steps. Let's start with the code that recognizes the limitations.   The package I used today will be used like this from the corridor media pipe Num Pa when it is opened.   When open, we will have some control over the webcam with v, so when we use the media pipe to recognize the line, we have at most several   Can you recognize the hand? So I\u2019m going to do only one right now, so I\u2019m going to type in \u201Cwork\u201D   After that, there are 11 gestures for gesture gestures, and 2 minutes is because they collected data on these various gestures.   Now, if I show you the iron in advance, what it is is like this. If you do this, e2   Hey, yeah because you are gay in Korea, but you usually do this in Europe or the United States in other countries. It doesn't work best.   Looks like the train now lacks a bit of training data. 3 then 4 5 5 Yes Spider-Man and Lock, and then this   626 Did you do it all like this?   Then he collected data that can recognize gestures. Feed these gestures with Yee Lee for 2 weeks.   Then, pull out 5 and give it a rock on the left. Next, let\u2019s look at the weisser. So, if you pull out these 3 gestures, you join the outside.   I can make a machine, so I\u2019m going to use only three of these 11 gestures   0 minutes 2 weeks feeding, 5 times the number, 9 times this is the vp, so this is each of these rocks   Then, it's the paper scissors. 0 5 9 Index Seo Young-ok, rock scissors, and match the paper.   So I made it separately like ips. It will recognize the hand with a media pipe. This is the u part, where you can control the volume with your finger on the dot.   I did it in making. If you don't know this part, please refer to the video of the finger volume control.   Then we will use 2 models this time. We will use the gesture recognition model.   So, there is a file called gesture train csv in the data collected by 2 people.   Gesture train csv files are collected in the data folder like this. So, all the achievements are stored here.   Next time, the label was saved at the end. I'm sorry Yong-young. Feed me that week. The mask under the fist sometimes gathers at this time, and various data is stored all the time.   Yes 4 There are about 110 training data right now, but it\u2019s very little.   Well, if you want to improve the accuracy a lot, if you collect more of this data,   Angle it closer and collect the data like this by labeling it   I wear kd arrest in the open 10th place, and why k is called of the latest algorithm, so I use k DEARS dave knn to do the learning like this.   Well, one kl model has been well trained and will be in   In the use of video capture of 5 pc video, the wakeme image is long and then the webcam is 1 frame c of the image 1   All of these parts were the contents of the volume control, so I will skip it. Next, the part that calculates the angle of the most important part and recognizes the gesture   Remain so what if your hand a   So Eun-hee Son's watch So, if it's not the 5th period, if it's not the 5th period, I can recognize several improvements.   First, we use a gun to repeat it, and then we each have daily dots and red dots. The red dots are called joints, and this is xy jet.   I\u2019m going to store the coordinates so there are 21 at the bottom of this side, because x is caught, xy is jet,   Save 3 and become 21 commas 3 So this landmark is saved for each joint, and the xy jet coordinates of the landmark are stored under 2 sets.   Then 21 comma 3 that array is created   Oh, here we calculate the angle with each joint intro vector. If you go to the Media Pepe document, there are indexes on various eat burners.   So, if you look at it, it seems that in the end, this index was added to v1 and dyron indexed to v2.   So, put it back in v So if you get on a boat except for Yeongha, then if you get on the movie Gohee,   Next, subtract this, subtract this 3, subtract 3, subtract the person, subtract the 1 and 2, subtract the person who works, except the person who is true.   Get all the vectors for each joint. Then, take out the fifth Kwon Eun-young and Orang.   Give me this, a sweet girl like this, like this, get a vector for each joint   So let's normalize it Normalize 2 We need to use the length of each vector Now we get the length Euclid 1   It's a formula to get the distance, but dividing it by the length, it will be normal rise.   The basic vector will come out and the job vector will be larger. After that, I'm not sure about this equation, but all arc cosines of the product   It is said that the angle is 9, and now I staren formation and arc cosine.   We get the angle and store it in a variable called Angry, so if you know about this equation, please let us know in the comments.   So I change the value of the angry radians to the values \u200B\u200Bof all kinds of this text. Did you make a gesture model with a student?   4 Take the trained knn model and do in-plus. So, change it to over tire ray and change it to a plotter 32-bit format, and then the data   After changing it to k, the realist K is like a spy and finds the price of life. So the result is the first ex of the resort.   So the index is stored here, and if the index is between and this rock beam   If there is one after signing up, it will display the valve result and if you are going to go to the valve result, if you don't see that spiderman or ok all those things   If you want to mark 11 seasons, you can use the code below to display all gestures.   You can use it. Because our purpose is to make Gal Weber, yoga is equivalent to this idiot.   I will show only gestures. Then I use pc off 15f text as this   Whether you wrote it in g's days, or whether it was a child of Cain's family, write the essence in writing.   Finally, draw a function that draws a brand mark on the knuckles using the Media Pipe Eight Roman Landmark function, and then   The requirement now is to actually show the picture. This will end a single dot pie.   Now I have made a model that can be recognized by Sun's Guyber. 4 knn is very fast and I am very satisfied with it 4   Like this, it is more recognized than rock, scissors, and it makes me glance at it like this   Yes, this watch works well on the side and on the side like this   Right this time, I'm going to look at the dual dot file, but it's a game where two guys play a ball ball.   So, for dual radio waves, you have to implant two hands first, so Max the Men's is 2   He said to recognize up to two hands, and here at the ips resort, he\u2019s the result of this distribution, so he\u2019ll store the coordinates in the hand. So   Here, we consider the essence of whether the result of Zester is buy of 2g in addition to RPASS realt.   Save it and save the position of each damage 2 on the finger here. So if you have more than 2 hands, there are only two anyway, but first   If it's been made like this and has 2 or more elements 4, the class runs this code and when the first person hits the rock, the first day when the file is two   If the second person played the ball, it was said that the second person also drew the ball, so it\u2019s this because they saw the rock.   After that, the Japanese lost the scissors. So, there is a simple logic to determine whether or not the app is a cucumber.   Nougat now has a code like that to mark it in letters It's very simple I've been able to sign up and make a friend by doing this   When you run the dual dot file,   It\u2019s very easy like this   Today, we learned how to make a rock, paper, scissors game using the Media Pipe Hand Landmark Detection Model.   Like this, I would have tried to make black ginseng using the image of a guy valve for noodles.   More than I think, we can make a model that recognizes 11 gestures with a simple algorithm such as kn using angles on the knuckles.   It\u2019s very accurate. Then I\u2019ll learn that you made a model. If you enjoyed it, please subscribe and like it. Next time, it\u2019s more interesting and interesting.   I'll see you over the project. Bye" #@param {type:"string"}
do_you_want__original_video_background_music= "No"#@param ["Yes","No"]
if do_you_want__original_video_background_music== "Yes":
  remove_list=["add_music_on_audio.mp3","speed_match.mp3","tts_audio.mp3","vocals.wav","accompaniment.wav",f"output{extension}"]
  for i in remove_list:
    try:
      os.remove(i)
    except:
      pass
  #extract audio
  os.system(f"ffmpeg -i input_video{extension} input_audio.mp3")
  bash_f='spleeter separate -p spleeter:2stems -o output input_audio.mp3'
  split_music_and_voice = os.system(bash_f)
  if split_music_and_voice ==0:
    voice=f'{parent_directory}/input_video/output/input_audio/vocals.wav'
    shutil.move(voice,".")
    music=f'{parent_directory}/input_video/output/input_audio/accompaniment.wav'
    shutil.move(music,".")
    os.system("rm -rf ./output")
    os.system("rm -rf ./pretrained_models")
else:
  pass


subtitles_language= "English" #@param ["English","korean", "Hindi", "Bengali", "Chinese", "Japanese"]

if subtitles_language == "English":
    in_lang = "en"
elif subtitles_language == "korean":
    in_lang = "ko"    
elif subtitles_language == "Hindi":
    in_lang = "hi"
elif subtitles_language == "Bengali":
    in_lang = "bn"
elif subtitles_language == "Chinese":
    in_lang = "zh-cn"
elif subtitles_language == "Japanese":
    in_lang = "ja"


english_accent ="Default" #@param ["Default","India","United Kingdom","United States","Canada","Australia","Ireland","South Africa"]

if english_accent == "Default":
    tld = "com"
elif english_accent == "India":
    tld = "co.in"

elif english_accent == "United Kingdom":
    tld = "co.uk"
elif english_accent == "United States":
    tld = "com"
elif english_accent == "Canada":
    tld = "ca"
elif english_accent == "Australia":
    tld = "com.au"
elif english_accent == "Ireland":
    tld = "ie"
elif english_accent == "South Africa":
    tld = "co.za"



tts = gTTS(paste_subtitles, lang=in_lang, tld=tld, slow=False)
tts.save(f"tts_audio.mp3")
audio = AudioSegment.from_file("tts_audio.mp3")
audio_duration=audio.duration_seconds
a_float =audio_duration/video_duration
formatted_float = "{:.2f}".format(a_float)
sound = AudioSegment.from_file("tts_audio.mp3")

def speed_change(sound, speed=1.0):
    # Manually override the frame_rate. This tells the computer how many
    # samples to play per second
    sound_with_altered_frame_rate = sound._spawn(sound.raw_data, overrides={
         "frame_rate": int(sound.frame_rate * speed)
      })
     # convert the sound with altered frame rate to a standard frame rate
     # so that regular playback programs will work right. They often only
     # know how to play audio at standard frame rate (like 44.1k)
    return sound_with_altered_frame_rate.set_frame_rate(sound.frame_rate)


#synchronize_video_and_audio_time can change your pitch it sound may be bad
synchronize_video_and_audio_time = "Yes"#@param ["Yes","No"]  
if synchronize_video_and_audio_time == "Yes":
  match_video_speed_with_audio = speed_change(sound,float(formatted_float))
  match_video_speed_with_audio.export('speed_match.mp3', format = 'mp3')


overlay_audio=""
if do_you_want__original_video_background_music =="Yes" and split_music_and_voice ==0:
  if synchronize_video_and_audio_time == "Yes":
    raw_audio="speed_match.mp3"
  elif synchronize_video_and_audio_time == "No":
    raw_audio="input_audio.mp3"
  sound1 = AudioSegment.from_mp3(raw_audio)
  sound2 = AudioSegment.from_mp3("accompaniment.wav")
  # mix sound2 with sound1, starting at 5000ms into sound1)
  output = sound1.overlay(sound2, position=5000)
  # save the result
  output.export("add_music_on_audio.mp3", format="mp3")
  overlay_audio="add_music_on_audio.mp3"
elif do_you_want__original_video_background_music =="No":
  overlay_audio="tts_audio.mp3"
  if synchronize_video_and_audio_time == "Yes":
    overlay_audio = "speed_match.mp3"


#add tts voice on video
var=os.system(f"ffmpeg -i input_video{extension} -i {overlay_audio} -c:v copy -c:a aac -map 0:v:0 -map 1:a:0 output{extension}")    


export_file_name=subtitles_language+"-"+the_video.replace(youtube_id[-1],"")
try:
  os.remove(f"{parent_directory}/output_video/{export_file_name}")
except:
  pass
try:
  shutil.copy(f"output{extension}",f"{parent_directory}/output_video/{export_file_name}")
except Exception as e:
  print(e)
if var ==0:  
  print("Your video location is: ")
  print(f"{parent_directory}/output_video/{export_file_name}")
  download_path=f"{parent_directory}/output_video/{export_file_name}"
# ipd.Audio("/content/input_video/tts_audio.mp3")
#audio notification
successful()

Your video location is: 
/content/output_video/English-가위바위보 기계 만들기 - 손가락 인식 인공지능-.mp4


In [11]:
# @title ##**Download translate video** { display-mode: "form" }

from google.colab import files
files.download(download_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
# @title ##**Display translate video** { display-mode: "form" }

from kora.drive import upload_public
from IPython.display import HTML
url = upload_public(download_path)
clear_output()
HTML(f"""<video src={url} width=500 controls/>""")